# 🤝 문서 유사도 (Document Similarity): 내 영혼의 단짝 찾기

이전 시간에 배운 **코사인 유사도**를 실전 데이터에 적용해서, **"추천 시스템"**을 만들어볼 거야.

> **미션**: 어떤 호텔 리뷰를 하나 골랐을 때, 그와 가장 비슷한 내용을 담은 리뷰들을 찾아라!

---

## 1. 데이터 준비 (복습!)
02번 파일에서 했던 것처럼 데이터를 불러오고 벡터화할 거야. 
(설명은 02번을 참고해줘! 여기선 빠르게 지나갈게 슝=3)

In [ ]:
import pandas as pd
import glob, os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 1. 데이터 로드
path = './data/OpinosisDataset1.0/topics'
all_files = glob.glob(os.path.join(path, '*.data'))

filename_list = []
opinion_text = []

for file in all_files:
    df = pd.read_table(file, index_col=None, header=None, encoding='latin1')
    filename_ = file.split('/')[-1]
    filename = filename_.split('.')[0]
    filename_list.append(filename)
    opinion_text.append(df.to_string(header=False, index=False))

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})

# 2. 전처리 함수 (어근 추출)
def LemNormalize(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    tokens = nltk.word_tokenize(text)
    lemmar = WordNetLemmatizer()
    return [lemmar.lemmatize(token, pos='v') for token in tokens]

# 3. TF-IDF 벡터화
tfidf_vect = TfidfVectorizer(
    tokenizer=LemNormalize
    stop_words='english',
    ngram_range=(1,2),
    min_df=0.05,
    max_df=0.85
)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])
print("준비 완료! 행렬 크기:", feature_vect.shape)

---

## 2. 유사도 계산: 모든 문서끼리 비교하기 ⚔️

51개의 문서가 서로 얼마나 닮았는지 계산해볼게.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 모든 문서(feature_vect)와 모든 문서(feature_vect)를 비교!
# 결과는 51x51 크기의 표(행렬)가 될 거야.
pair_similarity = cosine_similarity(feature_vect, feature_vect)

print(pair_similarity.shape)
print(pair_similarity)

---

## 3. 추천 시스템 구현: 이 리뷰와 비슷한 건? 🕵️‍♀️

이제 본격적으로 추천을 해보자. 
예를 들어, **"bathroom_bestwestern_hotel_sfo"** (샌프란시스코 베스트웨스턴 호텔 화장실) 리뷰를 보고 있는 사람에게,
다른 비슷한 호텔 리뷰를 추천해주고 싶어.

In [ ]:
# 1. 우리가 기준으로 삼을 리뷰 찾기
# 파일명에 'bathroom_bestwestern'이 들어가는 문서 찾기
target_idx = document_df[document_df['filename'].str.contains('bathroom_bestwestern')].index[0]

print(f"기준 문서 인덱스: {target_idx}")
print(f"파일명: {document_df.iloc[target_idx]['filename']}")

In [ ]:
import numpy as np

# 2. 이 문서(target_idx)와 다른 모든 문서 간의 유사도 가져오기
my_similarity = pair_similarity[target_idx]

# 3. 유사도가 높은 순서대로 정렬 (argsort 활용)
# argsort()는 값을 정렬하는 게 아니라, '정렬된 인덱스'를 반환해.
# 근데 기본이 오름차순(작은거 -> 큰거)이라서, [::-1]로 뒤집어서 내림차순(큰거 -> 작은거)으로 만들어야 해.
sorted_idx = my_similarity.argsort()[::-1]

# 4. 자기 자신(1등)은 제외하고, 2등부터 6등까지만 뽑아보자 (Top 5)
top_5_idx = sorted_idx[1:6]

print("가장 비슷한 리뷰 Top 5:")
print(document_df.iloc[top_5_idx]['filename'])

### 🔎 결과 분석
결과를 보면 **"room_holiday_inn"**, **"rooms_swissotel"** 처럼 
다른 호텔의 **방/화장실** 관련 리뷰들이 추천된 걸 볼 수 있어.

즉, 컴퓨터가 내용만 보고도 **"아, 이 사람은 호텔 시설에 관심이 있구나!"** 라고 파악한 거지. 대단하지? 🎉

---

## 🎓 마무리 퀴즈

**Q. `cosine_similarity(A, B)`의 결과값이 1에 가까울수록 두 문서는?**
1. 매우 비슷하다.
2. 전혀 관계 없다.
3. 반대 내용이다.

<details>
<summary>정답 확인</summary>

**1번 매우 비슷하다**

</details>